In [0]:
!pip install pytorch_pretrained_bert
!pip install sentencepiece
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertAdam, BertForSequenceClassification
from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
import re
import torch
from pytorch_pretrained_bert.tokenization import BertTokenizer
import os

from nltk.corpus import stopwords
import nltk
!pip install contractions
import contractions
import unicodedata
import re
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

     |████████████████████████████████| 133kB 2.7MB/s 
     |████████████████████████████████| 1.0MB 2.7MB/s 


Using TensorFlow backend.


     |████████████████████████████████| 317kB 4.0MB/s 
     |████████████████████████████████| 245kB 57.7MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.0-cp36-cp36m-linux_x86_64.whl size=81699 sha256=e4784a33b68e0b08fb8fa61573cdfe52e56364c4a0c5c593c730347566419154
  Stored in directory: /root/.cache/pip/wheels/0a/90/61/87a55f5b459792fbb2b7ba6b31721b06ff5cf6bde541b40994
Successfully built pyahocorasick
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [0]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

'Tesla P100-PCIE-16GB'

In [0]:
train = pd.read_csv('/content/drive/My Drive/Janatha NLP Hackathon/train.csv')
game = pd.read_csv('/content/drive/My Drive/Janatha NLP Hackathon/game_overview.csv')
test = pd.read_csv('/content/drive/My Drive/Janatha NLP Hackathon/test.csv')

In [0]:
def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return new_words

def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words

def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        new_word = re.sub(r'(.)\1+', r'\1\1', new_word) 
        if new_word != '':
            new_words.append(new_word)
    return new_words

def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    new_words = []
    for word in words:
        if word not in stopwords.words('english'):
            new_words.append(word)
    return new_words

def lemmatize_verbs(words):
    """Lemmatize verbs in list of tokenized words"""
    lemmatizer = WordNetLemmatizer()
    lemmas = []
    for word in words:
        lemma = lemmatizer.lemmatize(word, pos='v')
        lemmas.append(lemma)
    return lemmas

def replace_contractions(text):
    """Replace contractions in string of text"""
    return contractions.fix(text)


def removespace(text): #remove user name, special characters and urls
    # replacing the special characters and numbers
    text = re.sub(r'([^A-Za-z0-9\s])',' ',text)
    #replacing multiple spaces in to a single space
    text = re.sub(' +',' ', text)
    return text

In [0]:
def preprocess(text):
    text = removespace(text)
    text = replace_contractions(text)
    content = nltk.word_tokenize(text)
    words = remove_non_ascii(content)
    words = to_lowercase(words)
    # words = lemmatize_verbs(words)
    words = remove_punctuation(words)
    # words = remove_stopwords(words)
    # text= ' '.join(list(dict.fromkeys(words)))
    text = ' '.join(words)
    return text

In [0]:
train['title'] = train['title'].apply(preprocess)
game['title'] = game['title'].apply(preprocess)
test['title'] = test['title'].apply(preprocess)

game['tags'] = game['tags'].apply(preprocess)
game['overview'] = game['overview'].apply(preprocess)

In [0]:
train['proc_review'] = train['user_review'].apply(preprocess)
test['proc_review'] = test['user_review'].apply(preprocess)

In [0]:
train_df = pd.merge(train, game, on='title', how='left')
test_df = pd.merge(test, game, on='title', how='left')

In [0]:
train_df.isna().sum()

review_id            0
title                0
year               178
user_review          0
user_suggestion      0
proc_review          0
developer            0
publisher            0
tags                 0
overview             0
dtype: int64

input token ids is tokenizer converts tokens using vocab file. <br>
input masks are either 0 or 1. 1 for useful tokens, 0 for padding.<br>
segment ids are either 0 or 1. For 2 text training: 0 for the first one, 1 for the second one.

MAx sequence length is choosen as 128 based on the available text length in user review

In [0]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
x_train, x_valid, y_train, y_valid = train_test_split(train_df, train_df['user_suggestion'], 
                                                      random_state=0,test_size=0.1, stratify=train_df['user_suggestion'])

# BERT TRY in pytorch

In [0]:
BERT_MODEL = 'bert-base-cased'
MAX_SEQ_LEN = 512
TRAIN_BATCH_SIZE=6
EVAL_BATCH_SIZE = 6
CACHE_DIR = './'
NUM_LABELS = 2
# num_labels=2
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 2
GRADIENT_ACCUMULATION_STEPS = 1
WARMUP_PROPORTION = 0.1

In [0]:
from pytorch_pretrained_bert import BertTokenizer

tokenizer = BertTokenizer.from_pretrained(BERT_MODEL, do_lower_case=False)


100%|██████████| 213450/213450 [00:00<00:00, 386846.66B/s]


In [0]:
def get_ids(tokens, tokenizer, max_seq_length):
    """Token ids from Tokenizer vocab"""
    token_ids = tokenizer.convert_tokens_to_ids(tokens,)
    input_ids = token_ids + [0] * (max_seq_length-len(token_ids))
    return input_ids

def get_masks(tokens, max_seq_length):
    return [1]*len(tokens) + [0] * (max_seq_length - len(tokens))
 
def get_segments(tokens, max_seq_length):
    """Segments: 0 for the first sequence, 1 for the second"""
    segments = []
    current_segment_id = 0
    for token in tokens:
        segments.append(current_segment_id)
        if token == "[SEP]":
            current_segment_id = 1
    return segments + [0] * (max_seq_length - len(tokens))

In [0]:
def create_single_input(sentence,MAX_LEN):
  
  stokens = tokenizer.tokenize(sentence[0])
  tags = tokenizer.tokenize(sentence[1])
  
  stokens = stokens[:254]
  tags = tags[:255]
  
  stokens = ["[CLS]"] + stokens + ["[SEP]"] + tags + ["[SEP]"]
 
  ids = get_ids(stokens, tokenizer, MAX_SEQ_LEN)
  masks = get_masks(stokens, MAX_SEQ_LEN)
  segments = get_segments(stokens, MAX_SEQ_LEN)
 
  return ids,masks,segments
 
def create_input_array(sentences,tags):
 
  input_ids, input_masks, input_segments = [], [], []
 
  for sentence in tqdm_notebook(zip(sentences,tags),position=len(sentences), leave=True):
  
    ids,masks,segments=create_single_input(sentence,MAX_SEQ_LEN-3)
 
    input_ids.append(ids)
    input_masks.append(masks)
    input_segments.append(segments)
 
  return [np.asarray(input_ids, dtype=np.int32), 
            np.asarray(input_masks, dtype=np.int32), 
            np.asarray(input_segments, dtype=np.int32)]

In [0]:
x_train.columns

Index(['review_id', 'title', 'year', 'user_review', 'user_suggestion',
       'proc_review'],
      dtype='object')

In [0]:
train_sentences = x_train['proc_review'].values
valid_sentences = x_valid['proc_review'].values
test_sentences = test['proc_review'].values

train_tags = x_train['tags'].values
valid_tags = x_valid['tags'].values
test_tags = test_df['tags'].values

train_labels = y_train.values
test_labels = [0]*test.shape[0]
valid_labels = y_valid.values

In [0]:
train_inputs = create_input_array(train_sentences, train_tags)
valid_inputs = create_input_array(valid_sentences, valid_tags)
test_inputs = create_input_array(test_sentences, test_tags)

In [0]:
# test_inputs = create_input_array(test_sentences)

100%|██████████| 8045/8045 [00:14<00:00, 567.01it/s]


In [0]:
all_input_ids = torch.tensor([list(f) for f in train_inputs[0]], dtype=torch.long)
all_input_mask = torch.tensor([list(f) for f in train_inputs[1]], dtype=torch.long)
all_segment_ids = torch.tensor([list(f) for f in train_inputs[2]], dtype=torch.long)
all_label_ids = torch.tensor([f for f in y_train], dtype=torch.long)

In [0]:
train_dataset = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)
train_sampler = RandomSampler(train_dataset)
train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=TRAIN_BATCH_SIZE)

In [0]:
all_input_ids = torch.tensor([list(f) for f in valid_inputs[0]], dtype=torch.long)
all_input_mask = torch.tensor([list(f) for f in valid_inputs[1]], dtype=torch.long)
all_segment_ids = torch.tensor([list(f) for f in valid_inputs[2]], dtype=torch.long)
all_label_ids = torch.tensor([f for f in y_valid], dtype=torch.long)

In [0]:
valid_dataset = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)
valid_sampler = SequentialSampler(valid_dataset)
valid_dataloader = DataLoader(valid_dataset, sampler=valid_sampler, batch_size=EVAL_BATCH_SIZE)

In [0]:
all_input_ids = torch.tensor([list(f) for f in test_inputs[0]], dtype=torch.long)
all_input_mask = torch.tensor([list(f) for f in test_inputs[1]], dtype=torch.long)
all_segment_ids = torch.tensor([list(f) for f in test_inputs[2]], dtype=torch.long)
all_label_ids = torch.tensor([[f] for f in test_labels], dtype=torch.long)

In [0]:
test_dataset = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)
test_sampler = SequentialSampler(test_dataset)
test_dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size=EVAL_BATCH_SIZE)

In [0]:
# Load pre-trained model (weights)
model = BertForSequenceClassification.from_pretrained(BERT_MODEL, cache_dir=CACHE_DIR, num_labels=NUM_LABELS)

In [0]:
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
   

In [0]:
!pip install transformers
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup

In [0]:
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]

In [0]:
total_steps = len(train_dataloader)*NUM_TRAIN_EPOCHS

In [0]:
num_train_optimization_steps = int(
    len(x_train) / TRAIN_BATCH_SIZE / GRADIENT_ACCUMULATION_STEPS) * NUM_TRAIN_EPOCHS

In [0]:
# optimizer = BertAdam(optimizer_grouped_parameters,
#                      lr=LEARNING_RATE,
#                      warmup=WARMUP_PROPORTION,
#                      t_total=num_train_optimization_steps)

In [0]:
optimizer = AdamW(optimizer_grouped_parameters,
                  lr = LEARNING_RATE,
                  )
# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)


In [0]:
global_step = 0
nb_tr_steps = 0
tr_loss = 0

In [0]:
print("  Num examples = ", len(x_train))
print("  Batch size = ", TRAIN_BATCH_SIZE)
print("  Num steps = ", total_steps)

  Num examples =  15744
  Batch size =  6
  Num steps =  5248


In [0]:
weights = [0.75,1.00]
class_weights = torch.FloatTensor(weights).to(device)

In [0]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))


import random
import numpy as np

# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128

# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# We'll store a number of quantities such as training and validation loss, 
# validation accuracy, and timings.
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()


In [0]:
# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)


In [0]:
def cal_eval_metrics(predictions,true_labels,epochval):
  flat_predictions = np.concatenate(predictions, axis=0)
  flat_predictions = np.argmax(flat_predictions, axis=1).flatten()
  flat_true_labels = np.concatenate(true_labels, axis=0)
  mcc = matthews_corrcoef(flat_true_labels, flat_predictions)
  print("Mcc and classification report for Epoch :",epochval)
  print(classification_report(flat_true_labels, flat_predictions))
  print(confusion_matrix(flat_true_labels, flat_predictions))
  print('Total MCC: %.3f' % mcc)

https://mccormickml.com/2019/07/22/BERT-fine-tuning/#41-bertforsequenceclassification

In [0]:
epochs = int(NUM_TRAIN_EPOCHS)
num_labels = 2
# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')
    predictions , true_labels = [], []
    predictions_train , true_labels_train = [], []

    # Measure how long the training epoch takes.
    t0 = time.time()
    total_train_accuracy=0
    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(tqdm_notebook(train_dataloader, desc="Iteration")):
        
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
            batch = tuple(t.to(device) for t in batch)
            b_input_ids, b_input_mask, b_segment_ids, b_label_ids = batch
        
        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
            model.zero_grad()        
            logits = model(b_input_ids, token_type_ids=b_segment_ids, attention_mask=b_input_mask,labels=None)

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
            loss_fct = CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, num_labels), b_label_ids.view(-1))
            total_train_loss += loss.item()
            logits_train = logits.detach().cpu().numpy()
            label_ids_train = b_label_ids.to('cpu').numpy()
        
        
            predictions_train.append(logits_train)
            true_labels_train.append(label_ids_train)
        
        
            # Calculate the accuracy for this batch of test sentences, and
            # accumulate it over all batches.
            total_train_accuracy += flat_accuracy(logits_train, label_ids_train)
            
            
            
        # Perform a backward pass to calculate the gradients.
            loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
            optimizer.step()
            # optimizer.zero_grad()
        # Update the learning rate.
            scheduler.step()

    # Report the final accuracy for this training run.
    avg_train_accuracy = total_train_accuracy / len(train_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_train_accuracy))

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_mcc = 0
    total_eval_loss = 0
    nb_eval_steps = 0
    
    for batch in tqdm_notebook(valid_dataloader, desc="EVALUATING"):
     
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_segment_ids, b_label_ids = batch
        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            # Get the "logits" output by the model. The "logits" are the output
            # values prior to applying an activation function like the softmax.
            logits = model(b_input_ids, 
                                   token_type_ids=b_segment_ids, 
                                   attention_mask=b_input_mask,
                                   labels=None)
            
        # Accumulate the validation loss.
        loss_fct = CrossEntropyLoss(weight=class_weights)
        loss = loss_fct(logits.view(-1, num_labels), b_label_ids.view(-1))
            
        total_eval_loss += loss.item()

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_label_ids.to('cpu').numpy()
        
        
        predictions.append(logits)
        true_labels.append(label_ids)
        
        
        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        total_eval_accuracy += flat_accuracy(logits, label_ids)
        
    cal_eval_metrics(predictions,true_labels,epoch_i+1)
    
    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(valid_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))
    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(valid_dataloader)
    
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)
    
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Train Accur.' : avg_train_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )


print("")
print("Training complete!")
print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))



======== Epoch 1 / 2 ========
Training...



  Accuracy: 0.95

  Average training loss: 0.18
  Training epcoh took: 0:17:57

Running Validation...



Mcc and classification report for Epoch : 1
              precision    recall  f1-score   support

           0       0.91      0.90      0.90       753
           1       0.92      0.93      0.93       997

    accuracy                           0.92      1750
   macro avg       0.92      0.91      0.91      1750
weighted avg       0.92      0.92      0.92      1750

[[676  77]
 [ 69 928]]
Total MCC: 0.830
  Accuracy: 0.92
  Validation Loss: 0.34
  Validation took: 0:00:36

======== Epoch 2 / 2 ========
Training...



  Accuracy: 0.96

  Average training loss: 0.18
  Training epcoh took: 0:17:57

Running Validation...



Mcc and classification report for Epoch : 2
              precision    recall  f1-score   support

           0       0.91      0.90      0.90       753
           1       0.92      0.93      0.93       997

    accuracy                           0.92      1750
   macro avg       0.92      0.91      0.91      1750
weighted avg       0.92      0.92      0.92      1750

[[676  77]
 [ 69 928]]
Total MCC: 0.830
  Accuracy: 0.92
  Validation Loss: 0.34
  Validation took: 0:00:36

Training complete!
Total training took 3:36:33 (h:mm:ss)


In [0]:
#Testing

model.eval()

# Tracking variables 
predictions_test , true_labels_test = [], []

 
for batch in tqdm_notebook(test_dataloader):
  # Add batch to GPU
  batch = tuple(t.to(device) for t in batch)
  
  # Unpack the inputs from our dataloader
  b_input_ids, b_input_mask, b_segment_ids, b_label_ids = batch
  
  # Telling the model not to compute or store gradients, saving memory and 
  # speeding up prediction
  with torch.no_grad():
      # Forward pass, calculate logit predictions
      logits = model(b_input_ids, token_type_ids=b_segment_ids, attention_mask=b_input_mask,labels=None)

  
  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  label_ids = b_label_ids.to('cpu').numpy()
  
  # Store predictions and true labels
  predictions_test.append(logits)
  true_labels_test.append(label_ids)

print('    DONE.')


    DONE.


In [0]:
flat_predictions = np.concatenate(predictions_test, axis=0)
flat_predictions = np.argmax(flat_predictions, axis=1).flatten()

In [0]:
test['user_suggestion'] = flat_predictions

In [0]:
test[['review_id','user_suggestion']].to_csv('bert_submission2.csv', index=False)

In [0]:
def save_checkpoint():
  checkpoint = {'model':model}
  torch.save(checkpoint,'/content/drive/My Drive/Janatha NLP Hackathon/bert_model.pt')

In [0]:
# save_checkpoint()